## 1. Import Libraries

In [6]:
# Feature_Engineering
from pyspark.sql.types import *
import pyspark.sql.functions as F
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import *
from pyspark.sql.functions import col, count, sum
# Modeling
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn import linear_model, datasets
from sklearn.metrics import *
from sklearn.model_selection import cross_val_score
import numpy as np
from pyspark.ml.feature import OneHotEncoder

## 2. Load Data and Convert to Spark Data Frame

In [7]:
# read files
sc.textFile("loan.csv").take(1)

[u'id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_il_6m,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m']

In [8]:
# load data as dataframe
loan_df=spark.read.csv("loan.csv",header=True)

In [9]:
loan_df.rdd.getNumPartitions()
type(loan_df)

pyspark.sql.dataframe.DataFrame

## 3. Create response variable and features
### 3.1 Remove some columns based on EDA results

In [10]:
# loan_df1 = loan_df.drop('desc','mths_since_last_delinq','mths_since_last_record','next_pymnt_d',
#                         'mths_since_last_major_derog','annual_inc_joint','dti_joint','verification_status_joint',
#                         'open_acc_6m','open_il_6m','open_il_12m','open_il_24m','mths_since_rcnt_il','total_bal_il',
#                         'il_util','open_rv_12m','open_rv_24m','max_bal_bc','all_util','inq_fi','total_cu_tl',
#                         'inq_last_12m', # with a lot NA
#                         'id','member_id','collection_recovery_fee','last_pymnt_amnt','last_pymnt_d','out_prncp','out_prncp_inv',
#                         'pymnt_plan','recoveries','term','title','total_pymnt','total_pymnt_inv','total_rec_int',
#                         'total_rec_late_fee','total_rec_prncp','url','verification_status', 'initial_list_status', 
#                         'last_credit_pull_d','policy_code','emp_title','last_credit_pull_d' # domain knowledge
#                        )

#is that intentionally not leaving out drops?

In [11]:
# print loan_df1.head(1)
# print len(loan_df1.columns)

In [12]:
loan_df2=loan_df.select(
    loan_df.loan_amnt.cast("float"),
    loan_df.funded_amnt.cast("float"),
    loan_df.funded_amnt_inv.cast("float"),
    loan_df.int_rate.cast("float"),  #convert to float
    loan_df.installment.cast("float"), #convert to float
    'grade',
    'sub_grade',   # NEED TO BE DUMMIED
    'emp_length',
    'home_ownership', #group & make it dummy, done 
    loan_df.annual_inc.cast("float"),
    # 'issue_d',  #why comment out?
    'loan_status', 
    # response variable (pending for change after group discussion)
    'purpose',  #make it dummy
#     'zip_code', #interesting, worth dummying, might run into problems tho
#     'addr_state', #dummy
    'verification_status',
#     'initial_list_status',
    loan_df.dti.cast("float"),#float done
    loan_df.delinq_2yrs.cast("integer"),  
    #take out NAs, 2? 1, 3, mostly 0
    # 'earliest_cr_line',
    loan_df.inq_last_6mths.cast("integer"), #similar to delingq_2yr
    loan_df.open_acc.cast("integer"), 
    loan_df.pub_rec.cast("integer"), #what is it? 0,1
    loan_df.revol_bal.cast("float"), #done
    loan_df.revol_util.cast("float"),#done
    loan_df.total_acc.cast("integer"),
    #loan_df.last_credit_pull_d.cast("integer"), #date
    loan_df.acc_now_delinq.cast("integer"),#ok
    loan_df.tot_coll_amt.cast("float"), #done
    loan_df.tot_cur_bal.cast("float"), #done
    loan_df.total_rev_hi_lim.cast("integer") #what is this
)

### 3.2 Create response variable and remove rows with no valid response variable 

In [13]:
# needs to fix that, encoding doesn't seem to be right

def whetherpaid(x):
    if x in ['Default', 'Charged Off', 'Does not meet the credit policy. Status:Charged Off']:
        return 1
    elif x in ['Does not meet the credit policy. Status:Fully Paid', 'Fully Paid']:
        return 0
    else:
        return -1

In [14]:
paidflag = udf(lambda x: whetherpaid(x))

In [15]:
#subject to change
loan_df3 = loan_df2.withColumn(
    'paid_flag',
    paidflag('loan_status').cast("integer")
        ).where("paid_flag != -1").drop('loan_status')

In [16]:
loan_df3.printSchema()


root
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- grade: string (nullable = true)
 |-- sub_grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- purpose: string (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = true)
 |-- tot_cur_bal: float (nullable = true)
 |-- total_rev_hi_lim: integ

### 3.3 Add more features
#### 3.3.1 Create a numeric feature for "emp_length"

In [17]:
import re
def convert_to_int(s):
    s = re.sub('\\D', '', s)  #remove any non-digital character
    #\d matches any digital, #\D matches any non-digital
    try:
        return s
    except ValueError:
        return 'NaN'

emp_to_num = udf(convert_to_int)
loan_df4 = loan_df3.withColumn('emp_len',emp_to_num('emp_length').cast('integer')).drop('emp_length')

#### 3.3.2 Create numeric variable for grade 

In [18]:
# count how many nulls in each column
def count_null(c):
    return sum(col(c).isNull().cast("integer")).alias(c)

exprs = [count_null(c) for c in loan_df4.columns[0:9]]
loan_df4.agg(*exprs).show()

+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+
|loan_amnt|funded_amnt|funded_amnt_inv|int_rate|installment|grade|sub_grade|home_ownership|annual_inc|
+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+
|        0|          0|              0|       0|          0|    0|        0|             0|         4|
+---------+-----------+---------------+--------+-----------+-----+---------+--------------+----------+



#### 3.3.3 Create numeric variable for grade 

In [19]:
def home_ownership_func(x):
    if x in ['ANY','OTHER','NONE']:
        return 'Other'
    else: 
        return x

home_ownership = udf(home_ownership_func)  #fixed a function
loan_df5 = loan_df4.withColumn('home_ownership',home_ownership('home_ownership'))

In [20]:
loan_df5.select('home_ownership').groupBy('home_ownership').count().show()

+--------------+------+
|home_ownership| count|
+--------------+------+
|           OWN| 22282|
|         Other|   228|
|          RENT|107831|
|      MORTGAGE|126598|
+--------------+------+



#### 3.3.4 Add loan_inc_ratio feature 

In [21]:
def calculate_ratio(a, b):
    try:
        return a/float(b)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None

ratio = udf(calculate_ratio) #define a udf_function ratio

In [22]:
loan_df6 = loan_df5.withColumn(
    'loan_inc_ratio',ratio('loan_amnt','annual_inc'
                          ).cast('float'))

In [23]:
loan_df7 = loan_df6.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

#### 3.3.5 Add feature instal_inc_ratio


In [24]:
def calculate_monthly_ratio(a, b):
    try:
        return a/(float(b)/12)
    except TypeError:
        return None
    except ZeroDivisionError:
        return None
    
monthly_ratio = udf(calculate_monthly_ratio)

In [25]:
loan_df8 = loan_df7.withColumn(
    'instal_inc_ratio',
    ratio('installment','annual_inc').cast('float'))

#### 3.3.6 Create dummy variables
* Use StringIndexer to encode a string column of labels to a column of label indices, and most frequent label gets index 0.

* Use OneHotEncoder to convert indices into dummy vectors

In [26]:
convert_list = [      
               'purpose',
               'grade',
               'sub_grade',
               'verification_status',
               'home_ownership',
                ]

for item in convert_list:
    print item
    indexer = StringIndexer(inputCol=item, outputCol=item + 'Index')    
    loan_df8 = indexer.fit(loan_df8).transform(loan_df8).drop(item)
    onehotenc = OneHotEncoder(inputCol=item + 'Index', outputCol=item+"-onehot", dropLast=False)
    # we can experiment with True
    loan_df8 = onehotenc.transform(loan_df8).drop(item+'Index')
    print item

purpose
purpose
grade
grade
sub_grade
sub_grade
verification_status
verification_status
home_ownership
home_ownership


In [27]:
loan_df8 = loan_df8.fillna(0.0, ['tot_coll_amt','tot_cur_bal', 'total_rev_hi_lim'])
loan_df9 = loan_df8.dropna()

In [28]:
loan_df9.columns

['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'paid_flag',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
 'purpose-onehot',
 'grade-onehot',
 'sub_grade-onehot',
 'verification_status-onehot',
 'home_ownership-onehot']

In [39]:
loan_df_final = loan_df9.select(
 'paid_flag',
 'loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'emp_len',
 'loan_inc_ratio',
 'instal_inc_ratio',
 'purpose-onehot',
 'grade-onehot',
 'sub_grade-onehot',
 'verification_status-onehot',
 'home_ownership-onehot'
)

In [68]:
loan_df_final.take(1)
loan_df_final.printSchema()

root
 |-- paid_flag: integer (nullable = true)
 |-- loan_amnt: float (nullable = true)
 |-- funded_amnt: float (nullable = true)
 |-- funded_amnt_inv: float (nullable = true)
 |-- int_rate: float (nullable = true)
 |-- installment: float (nullable = true)
 |-- annual_inc: float (nullable = true)
 |-- dti: float (nullable = true)
 |-- delinq_2yrs: integer (nullable = true)
 |-- inq_last_6mths: integer (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: float (nullable = true)
 |-- revol_util: float (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- acc_now_delinq: integer (nullable = true)
 |-- tot_coll_amt: float (nullable = false)
 |-- tot_cur_bal: float (nullable = false)
 |-- total_rev_hi_lim: integer (nullable = true)
 |-- emp_len: integer (nullable = true)
 |-- loan_inc_ratio: float (nullable = true)
 |-- instal_inc_ratio: float (nullable = true)
 |-- purpose-onehot: vector (nullable = true)
 |-- grade-on

In [31]:
# count how many nulls in each column
# def count_null(c):
#     return sum(col(c).isNull().cast("integer")).alias(c)

# exprs = [count_null(c) for c in loan_df9.columns]
# loan_df9.agg(*exprs).toPandas().T

## 4. Modeling

### 4.1 Instantiating Data

In [32]:
# from pyspark.mllib.tree import RandomForest, RandomForestModel
# from pyspark.mllib.util import MLUtils
# from pyspark.mllib.regression import LabeledPoint

# # Load and parse the data file into an RDD of LabeledPoint.
# data = loan_df_final.rdd.map(lambda row: LabeledPoint(row[0], row[1:]))
# # Split the data into training and test sets (30% held out for testing)
# (trainingData, testData) = data.randomSplit([0.7, 0.3])

# model = RandomForest.trainClassifier(trainingData, numClasses=2, categoricalFeaturesInfo={},
#                                      numTrees=11, featureSubsetStrategy="auto",
#                                      impurity='gini', maxDepth=15, maxBins=32)

# # Evaluate model on test instances and compute test error
# predictions = model.predict(testData.map(lambda x: x.features))
# labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
# testErr = labelsAndPredictions.filter(lambda (v, p): v != p).count() / float(testData.count())
# print('Test Error = ' + str(testErr))

In [220]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier

from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.mllib.evaluation import MulticlassMetrics

In [80]:
 # Split the data into training and test sets (30% held out for testing)
loan_df=loan_df_final.withColumnRenamed('paid_flag','label')
loan_df.head(2)

[Row(label=0, loan_amnt=5000.0, funded_amnt=5000.0, funded_amnt_inv=4975.0, int_rate=10.649999618530273, installment=162.8699951171875, annual_inc=24000.0, dti=27.649999618530273, delinq_2yrs=0, inq_last_6mths=1, open_acc=3, pub_rec=0, revol_bal=13648.0, revol_util=83.69999694824219, total_acc=9, acc_now_delinq=0, tot_coll_amt=0.0, tot_cur_bal=0.0, total_rev_hi_lim=0, emp_len=10, loan_inc_ratio=0.2083333283662796, instal_inc_ratio=0.006786249577999115, purpose-onehot=SparseVector(265, {1: 1.0}), grade-onehot=SparseVector(7, {0: 1.0}), sub_grade-onehot=SparseVector(35, {3: 1.0}), verification_status-onehot=SparseVector(3, {0: 1.0}), home_ownership-onehot=SparseVector(4, {1: 1.0})),
 Row(label=1, loan_amnt=2500.0, funded_amnt=2500.0, funded_amnt_inv=2500.0, int_rate=15.270000457763672, installment=59.83000183105469, annual_inc=30000.0, dti=1.0, delinq_2yrs=0, inq_last_6mths=5, open_acc=3, pub_rec=0, revol_bal=1687.0, revol_util=9.399999618530273, total_acc=4, acc_now_delinq=0, tot_coll_a

In [81]:
va=VectorAssembler(outputCol='features',inputCols=loan_df.columns[:])
data=va.transform(loan_df).select('features','label')

In [82]:
data.head(3)

[Row(features=SparseVector(336, {1: 5000.0, 2: 5000.0, 3: 4975.0, 4: 10.65, 5: 162.87, 6: 24000.0, 7: 27.65, 9: 1.0, 10: 3.0, 12: 13648.0, 13: 83.7, 14: 9.0, 19: 10.0, 20: 0.2083, 21: 0.0068, 23: 1.0, 287: 1.0, 297: 1.0, 329: 1.0, 333: 1.0}), label=0),
 Row(features=SparseVector(336, {0: 1.0, 1: 2500.0, 2: 2500.0, 3: 2500.0, 4: 15.27, 5: 59.83, 6: 30000.0, 7: 1.0, 9: 5.0, 10: 3.0, 12: 1687.0, 13: 9.4, 14: 4.0, 19: 1.0, 20: 0.0833, 21: 0.002, 28: 1.0, 288: 1.0, 301: 1.0, 331: 1.0, 333: 1.0}), label=1),
 Row(features=SparseVector(336, {1: 2400.0, 2: 2400.0, 3: 2400.0, 4: 15.96, 5: 84.33, 6: 12252.0, 7: 8.72, 9: 2.0, 10: 2.0, 12: 2956.0, 13: 98.5, 14: 10.0, 19: 10.0, 20: 0.1959, 21: 0.0069, 27: 1.0, 288: 1.0, 304: 1.0, 330: 1.0, 333: 1.0}), label=0)]

In [83]:
data.show(3)

+--------------------+-----+
|            features|label|
+--------------------+-----+
|(336,[1,2,3,4,5,6...|    0|
|(336,[0,1,2,3,4,5...|    1|
|(336,[1,2,3,4,5,6...|    0|
+--------------------+-----+
only showing top 3 rows



In [75]:
# Creating Training and Test Data
data_sets=data.randomSplit([0.8,0.2])
data_train=data_sets[0].cache()
data_test=data_sets[1].cache()

### 4.2 Training the Data 

In [93]:
# Fit random forest
rf = RandomForestClassifier(labelCol="label", featuresCol="features",maxDepth=15)
rfmodel=rf.fit(data_train)

In [219]:
# Make predictions.
predictions = rfmodel.transform(data_test)
predictions.show(10)

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|(336,[0,1,2,3,4,5...|    1|[4.63200978186682...|[0.23160048909334...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[9.04460715706085...|[0.45223035785304...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[6.75234405914103...|[0.33761720295705...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[6.27942947355296...|[0.31397147367764...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[5.05801471763658...|[0.25290073588182...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[10.6186470338651...|[0.53093235169325...|       0.0|
|(336,[0,1,2,3,4,5...|    1|[6.82247387248063...|[0.34112369362403...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[5.38044604108332...|[0.26902230205416...|       1.0|
|(336,[0,1,2,3,4,5...|    1|[6.98734159310858...|[0.34936707965542...|       1.0|
|(336,[0,1,2,3,4

In [95]:
# Select (prediction, true label) and compute test error
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))


Test Error = 0.0140983


In [155]:
prediction_list = predictions.collect()

In [168]:
prediction_list[0]

Row(features=SparseVector(336, {0: 1.0, 1: 12000.0, 2: 12000.0, 3: 12000.0, 4: 12.99, 5: 404.27, 6: 42000.0, 7: 21.36, 8: 1.0, 9: 1.0, 10: 20.0, 11: 1.0, 12: 4544.0, 13: 21.6, 14: 48.0, 16: 111.0, 17: 20176.0, 18: 21000.0, 19: 1.0, 20: 0.2857, 21: 0.0096, 22: 1.0, 287: 1.0, 295: 1.0, 331: 1.0, 333: 1.0}), label=1, rawPrediction=DenseVector([4.632, 15.368]), probability=DenseVector([0.2316, 0.7684]), prediction=1.0)

In [211]:
prediction_list[0][1]

1

In [216]:
predictionAndLabels = []
for item in prediction_list:
    predictionAndLabels.append((float(item[4]), float(item[1])))

In [218]:
metrics = MulticlassMetrics(sc.parallelize(predictionAndLabels))
print metrics.confusionMatrix().toArray()
print metrics.precision(1)
print metrics.recall(1)
print metrics.accuracy

[[ 40409.      0.]
 [   684.   8192.]]
1.0
0.922938260478
0.986121537993
